In [1]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

In [2]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IMDB Dataset of 50K Movie Reviews/IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [4]:
import nltk
nltk.download('stopwords')
english_stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
def load_dataset():
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IMDB Dataset of 50K Movie Reviews/IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


In [7]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
4110     [this, obscure, de, sica, delivers, goods, and...
22208    [i, know, rest, guys, watch, steven, seagal, m...
35944    [so, giant, mantis, hanging, arctic, tired, es...
47630    [che, bad, movie, deserves, reputation, uninte...
46207    [first, started, watching, became, one, favour...
                               ...                        
4136     [poor, tobe, hopper, he, directed, time, horro...
47674    [starting, tearjerking, poem, images, american...
40250    [i, stumbled, upon, movie, accident, i, mean, ...
10509    [up, point, gentle, rain, movie, i, found, wor...
47181    [this, movie, one, time, favorites, i, think, ...
Name: review, Length: 40000, dtype: object 

5000     [not, movie, everyone, movie, top, i, lover, b...
13257    [now, i, realize, people, trash, young, warrio...
8870     [if, fan, slap, stick, terrible, writing, awfu...
6303     [first, i, watched, movie, showtime, called, s...
31591    [this, short, terrible, disappointment, by, fa...
 

In [8]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [9]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[    8  3598   775 ...     0     0     0]
 [    1    47   272 ...     0     0     0]
 [  145  1200 12363 ...     0     0     0]
 ...
 [    1  5175   590 ...    61    13  3853]
 [ 2530   128  4098 ...   112    73   793]
 [    8     3     5 ...     0     0     0]] 

Encoded X Test
 [[  153     3   200 ...     0     0     0]
 [  377     1   860 ...  8411 33086    25]
 [   57   234  3972 ...     0     0     0]
 ...
 [    1    41   286 ...   430   211  4515]
 [ 2554     2 20343 ...     0     0     0]
 [    1  2069   165 ...     0     0     0]] 

Maximum review length:  130


In [10]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 130, 32)           2956864   
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2,981,761
Trainable params: 2,981,761
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [12]:
import time

start_time = time.time()

model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

end_time = time.time()
total_time = end_time - start_time
print("Total training time:", total_time, "seconds")

Epoch 1/5
313/313 [==============================] - ETA: 0s - loss: 0.4471 - accuracy: 0.7631
Epoch 1: accuracy improved from -inf to 0.76310, saving model to models/LSTM.h5
313/313 [==============================] - 53s 161ms/step - loss: 0.4471 - accuracy: 0.7631
Epoch 2/5
313/313 [==============================] - ETA: 0s - loss: 0.2092 - accuracy: 0.9241
Epoch 2: accuracy improved from 0.76310 to 0.92410, saving model to models/LSTM.h5
313/313 [==============================] - 49s 155ms/step - loss: 0.2092 - accuracy: 0.9241
Epoch 3/5
313/313 [==============================] - ETA: 0s - loss: 0.1217 - accuracy: 0.9624
Epoch 3: accuracy improved from 0.92410 to 0.96237, saving model to models/LSTM.h5
313/313 [==============================] - 49s 156ms/step - loss: 0.1217 - accuracy: 0.9624
Epoch 4/5
313/313 [==============================] - ETA: 0s - loss: 0.0863 - accuracy: 0.9741
Epoch 4: accuracy improved from 0.96237 to 0.97408, saving model to models/LSTM.h5
313/313 [======

In [18]:
# Get predicted probabilities from the model
y_pred = model.predict(x_test, batch_size=128)

# Set a custom threshold
threshold = 0.3

# Convert probabilities to class labels based on threshold
y_pred_classes = (y_pred > threshold).astype(int)

# Calculate accuracy
true = 0
for i, y in enumerate(y_test):
    if y == y_pred_classes[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred_classes) - true))
print('Accuracy: {}'.format(true/len(y_pred_classes)*100))


79/79 [==============================] - 4s 52ms/step
Correct Prediction: 8638
Wrong Prediction: 1362
Accuracy: 86.38


In [19]:
loaded_model = load_model('models/LSTM.h5')

In [20]:
review = str(input('Movie Review: '))

Movie Review: Nothing was typical about this. Everything was beautifully done in this movie, the story, the flow, the scenario, everything. I highly recommend it for mystery lovers, for anyone who wants to watch a good movie!


In [21]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  Nothing was typical about this Everything was beautifully done in this movie the story the flow the scenario everything I highly recommend it for mystery lovers for anyone who wants to watch a good movie
Filtered:  ['nothing typical everything beautifully done movie story flow scenario everything i highly recommend mystery lovers anyone wants watch good movie']


In [22]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[  75  671  172 1211  129    3   14 2753 2587  172    1  445  282  703
  1793  157  402   32    9    3    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]]


In [23]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 [==============================] - 1s 574ms/step
[[0.9684353]]


In [24]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive
